# Submitting Requests for updating data directory

In [2]:
import datajoint as dj
from datetime import datetime
dj.conn()

Connecting root@localhost:3306


DataJoint connection (connected) root@localhost:3306

In [3]:
dj.config['display.limit'] = 30

In [4]:
from ephys import ingestion, acquisition, ephys

## A request to update the data directory for a session

#### Identify the ***SessionInsertRequest*** to request a path update

In [5]:
ingestion.SessionInsertRequest & 'input_full_path LIKE "F:%"' & 'username = "testuser"'

username,input_full_path full path of the input directory on the requesting machine (including drive),request_time,animal_name,session_note,tracking_system_setup user-readable identifier of this setup (e.g. 'room_1_setup'),experiment_room
testuser,F:\Moser\wireless\20180424_light,2019-10-14 17:43:15,24321,=BLOB=,None,None


In [6]:
session_request_key = (ingestion.SessionInsertRequest & 'input_full_path LIKE "F:%"' & 'username = "testuser"').fetch1('KEY')

#### Build a path update request

In [57]:
path_update_request = {'path_update_request_username': 'testuser', 
                       'path_update_request_time': datetime.now(),
                       'path_update_type': 'session',
                       'new_full_path': 'F:/Moser/wireless/20180424_light'}

#### Insert the request

In [58]:
with dj.conn().transaction:
    ingestion.PathUpdateRequest.insert1(path_update_request)
    ingestion.PathUpdateRequest.Session.insert1({**path_update_request, **session_request_key}, ignore_extra_fields=True)

In [9]:
ingestion.PathUpdateRequest()

path_update_request_time,path_update_request_username,path_update_type,new_full_path full path of the input directory on the requesting machine (including drive)
2020-02-12 12:14:27,testuser,session,F:/Moser/wireless/20180424_light


#### Process the request

In [10]:
ingestion.ProcessPathUpdateRequest.populate()

In [11]:
ingestion.ProcessPathUpdateRequest * ingestion.PathUpdateRequest

path_update_request_time,path_update_request_username,process_time,path_update_status,path_update_detail,path_update_type,new_full_path full path of the input directory on the requesting machine (including drive)
2020-02-12 12:14:27,testuser,2020-02-12 12:14:35,error,New path is identical to current path,session,F:/Moser/wireless/20180424_light


#### Try invalid path

In [12]:
path_update_request = {'path_update_request_username': 'testuser', 
                       'path_update_request_time': datetime.now(),
                       'path_update_type': 'session',
                       'new_full_path': 'F:\\Moser\\rich_neuropixels\\2018-07-03_19-10-39'}

In [13]:
with dj.conn().transaction:
    ingestion.PathUpdateRequest.insert1(path_update_request)
    ingestion.PathUpdateRequest.Session.insert1({**path_update_request, **session_request_key}, ignore_extra_fields=True)

In [14]:
ingestion.ProcessPathUpdateRequest.populate()

In [15]:
ingestion.ProcessPathUpdateRequest * ingestion.PathUpdateRequest

path_update_request_time,path_update_request_username,process_time,path_update_status,path_update_detail,path_update_type,new_full_path full path of the input directory on the requesting machine (including drive)
2020-02-12 12:14:27,testuser,2020-02-12 12:14:35,error,New path is identical to current path,session,F:/Moser/wireless/20180424_light
2020-02-12 12:14:40,testuser,2020-02-12 12:15:27,denied,Mismatch recording type: found neuropixels from the specified path (original recording type: neurologger),session,F:\Moser\rich_neuropixels\2018-07-03_19-10-39


## A request to update the data directory for a curated clustering

#### Identify the ***CuratedClusteringRequest*** to request a path update

In [16]:
ingestion.CuratedClusteringRequest * ingestion.InsertedCuratedClustering

animal_id,datasource_id,session_time start time of this session,curation_timestamps ensure complete datetime here to prevent collision (in the rare case of multiple people curate at the exact same datetime),insertion_time When this probe was inserted,group_name,time_window_name unique name for this window set,"cluster_param_name some unique name, ideally reflecting the method as well","probe_model nick name, or other user-friendly model name of this probe",electrode_config_id hash of the group and group_member (ensure uniqueness),sync_method sync method name,username,curator,curation_notes,curation_output_full_path
40bfa03a7a7018cc,0,2018-07-03 20:32:28,2018-04-25 10:25:23,2019-09-11 14:08:45,det_2d32fa1d32e79e336ffd073d7eae7b9b,full_b13d14f1ce57eab0,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,root,richarga,,F:\Moser\rich_neuropixels\2018-07-03_19-10-39
40bfa03a7a7018cc,0,2018-07-03 20:32:28,2018-04-25 10:25:23,2019-09-11 14:09:22,det_2d32fa1d32e79e336ffd073d7eae7b9b,full_b13d14f1ce57eab0,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,root,richarga,,F:\Moser\rich_neuropixels\2018-07-03_19-10-39
575882867c176190,0,2019-09-24 09:16:14,2019-09-24 13:37:20,2019-04-04 00:00:00,det_229a013f343836f8976421f523887bb9,full_dbd211aff6e7622a,default_mclust,4-tetrode-array,e9c6dac82a8bcb02825b0be07735a967,linear_interp,emiliera,emiliera,,N:\emiliera\LEC_Time\Virus_LEC+hippocamp_rec\86866\2019-09-24_09-15-42
575882867c176190,0,2019-09-24 09:33:50,2019-09-27 14:10:10,2019-04-04 00:00:00,det_82b5ff9fe7464161f05b9f9f7b9c4fe4,full_6b17f214a76ac1b8,default_mclust,4-tetrode-array,e9c6dac82a8bcb02825b0be07735a967,linear_interp,emiliera,emiliera,,N:\emiliera\LEC_Time\Virus_LEC+hippocamp_rec\86866\2019-09-24_09-33-17
61fd2ac184c13c73,0,2017-10-02 15:54:25,2017-10-02 16:27:27,2017-10-02 00:00:00,det_6c2d14fe4aebefa1e762acba7f48904a,full_21d442485e680998,default_tint,4-tetrode-array,657119cc9e911c2c8a536abce16264f0,linear_interp,davidcr,davidcr,,N:\davidcr\74956\02102017
724240a0a756020f,0,2018-04-24 10:25:23,2018-04-25 10:25:23,2018-03-15 12:00:00,det_0d7c5dd06d63c5f14395b914f7f803e7,full_2f298ff9d92b03c7,default_kilosort,8-tetrode-array,00784804f372476162c7d90e94d8dc3d,linear_interp,testuser,torgeirw,,F:\Moser\wireless\20180424_light\20180424_01_light
724240a0a756020f,0,2018-04-24 10:25:23,2018-04-25 10:25:23,2018-03-15 12:10:00,det_0d7c5dd06d63c5f14395b914f7f803e7,full_2f298ff9d92b03c7,default_kilosort,8-tetrode-array,00784804f372476162c7d90e94d8dc3d,linear_interp,testuser,torgeirw,,F:\Moser\wireless\20180424_light\20180424_01_light


In [23]:
ingestion.CuratedClusteringRequest & 'curation_output_full_path LIKE "F:%"' & 'username = "testuser"'

animal_id,datasource_id,session_time start time of this session,curation_timestamps ensure complete datetime here to prevent collision (in the rare case of multiple people curate at the exact same datetime),username,curator,curation_notes,curation_output_full_path
724240a0a756020f,0,2018-04-24 10:25:23,2018-04-25 10:25:23,testuser,torgeirw,,F:\Moser\wireless\20180424_light\20180424_01_light


In [24]:
clustering_request_key = (ingestion.CuratedClusteringRequest 
                          & 'curation_output_full_path LIKE "F:%"' & 'username = "testuser"').fetch1('KEY')

#### Build a path update request

In [59]:
path_update_request = {'path_update_request_username': 'testuser', 
                       'path_update_request_time': datetime.now(),
                       'path_update_type': 'single-sess clustering',
                       'new_full_path': 'F:/Moser/wireless/20180424_light'}

#### Insert the request

In [60]:
with dj.conn().transaction:
    ingestion.PathUpdateRequest.insert1(path_update_request)
    ingestion.PathUpdateRequest.SingleSessClustering.insert1({**path_update_request,
                                                              **clustering_request_key}, ignore_extra_fields=True)

#### Process the request

In [63]:
ingestion.ProcessPathUpdateRequest.populate()

`ephys_v2_ingestion`.`__inserted_session`: 1 items
The delete is pending within the ongoing transaction.


In [64]:
ingestion.ProcessPathUpdateRequest * ingestion.PathUpdateRequest

path_update_request_time,path_update_request_username,process_time,path_update_status,path_update_detail,path_update_type,new_full_path full path of the input directory on the requesting machine (including drive)
2020-02-12 12:14:27,testuser,2020-02-12 12:14:35,error,New path is identical to current path,session,F:/Moser/wireless/20180424_light
2020-02-12 12:14:40,testuser,2020-02-12 12:15:27,denied,Mismatch recording type: found neuropixels from the specified path (original recording type: neurologger),session,F:\Moser\rich_neuropixels\2018-07-03_19-10-39
2020-02-12 12:50:48,testuser,2020-02-12 12:52:35,denied,"Mismatch cluster method: found kilosort from the specified path (original recording type: kilosort,klustakwik,mclust,tint)",single-sess clustering,F:/Moser/neurologger/20180424_light
2020-02-12 13:02:15,testuser,2020-02-12 13:09:56,replaced,,single-sess clustering,F:/Moser/neurologger/20180424_light
2020-02-12 13:10:14,testuser,2020-02-12 13:12:46,replaced,,single-sess clustering,F:/Moser/wireless/20180424_light
2020-02-12 13:14:03,testuser,2020-02-12 13:14:57,replaced,,session,F:/Moser2/wireless/20180424_light
2020-02-12 13:14:12,testuser,2020-02-12 13:14:58,replaced,,single-sess clustering,F:/Moser2/wireless/20180424_light
2020-02-12 13:16:05,testuser,2020-02-12 13:16:42,current,,session,F:/Moser/wireless/20180424_light
2020-02-12 13:16:11,testuser,2020-02-12 13:16:43,current,,single-sess clustering,F:/Moser/wireless/20180424_light
